In [ ]:
!pip install tensorflow-text
!pip install tf-models-official
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

import pandas as pd
import numpy as np

     |████████████████████████████████| 4.3MB 5.1MB/s 
     |████████████████████████████████| 1.6MB 5.2MB/s 
     |████████████████████████████████| 215kB 23.8MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 358kB 22.5MB/s 
     |████████████████████████████████| 686kB 28.1MB/s 
     |████████████████████████████████| 645kB 35.6MB/s 
     |████████████████████████████████| 38.2MB 81kB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 1.2MB 46.4MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22258 sha256=f5197e45c4870b7b12d15d732aa1b8f55be3ce197f244fb24de2fb83186fc86a
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=77dc30336cca11799c8694d7877d9b89e7a55

In [ ]:
import os
!pip install tensorflow transformers

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

     |████████████████████████████████| 2.5MB 5.1MB/s 
     |████████████████████████████████| 901kB 50.2MB/s 
     |████████████████████████████████| 3.3MB 43.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]
AUX_COLUMNS = ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
TEXT_COLUMN = 'comment_text'
TARGET_COLUMN = 'target'
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

In [ ]:
train_df = pd.read_csv('/content/gdrive/MyDrive/job/Upskill/test2/train.csv')
test_df = pd.read_csv('/content/gdrive/MyDrive/job/Upskill/test2/test.csv')

In [ ]:
x_train = train_df[TEXT_COLUMN].astype(str)
y_train = train_df[TARGET_COLUMN].values
y_aux_train = train_df[AUX_COLUMNS].values
x_test = test_df[TEXT_COLUMN].astype(str)

In [ ]:
print(x_train)

0          This is so cool. It's like, 'would you want yo...
1          Thank you!! This would make my life a lot less...
2          This is such an urgent design problem; kudos t...
3          Is this something I'll be able to install on m...
4                       haha you guys are a bunch of losers.
                                 ...                        
1804869    Maybe the tax on "things" would be collected w...
1804870    What do you call people who STILL think the di...
1804871    thank you ,,,right or wrong,,, i am following ...
1804872    Anyone who is quoted as having the following e...
1804873    Students defined as EBD are legally just as di...
Name: comment_text, Length: 1804874, dtype: object


In [ ]:
for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
    train_df[column] = np.where(train_df[column] >= 0.5, True, False)


###Vanilla BERT model

In [ ]:
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1"
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3, Total size: 1.96MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'.


In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1, Total size: 39.74MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1'.


In [ ]:
text_test = ['This is a toxic comment']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_word_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [  101  2023  2003  1037 11704  7615   102     0     0     0     0     0]
Input Mask : [1 1 1 1 1 1 1 0 0 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1
Pooled Outputs Shape:(1, 256)
Pooled Outputs Values:[ 0.9880961   0.02632275  0.59086233 -0.41103482 -0.143692    0.9995551
 -0.9988162  -0.23873784 -0.86690146 -0.14589763  0.9908594  -0.12006074]
Sequence Outputs Shape:(1, 128, 256)
Sequence Outputs Values:[[-1.3261037  -1.9732889   1.956214   ...  0.5859899   0.56647485
  -2.044166  ]
 [-1.0313797  -1.0769088   1.1369966  ...  1.0221493   0.73639005
  -1.0397089 ]
 [-1.3347017  -2.0484319   2.062935   ...  0.32304773  1.2253817
  -0.5659097 ]
 ...
 [-0.49464116 -2.109041    0.9121566  ... -0.46682933  0.0539288
  -1.0628998 ]
 [-0.67919993 -2.1776354   1.5580871  ... -0.6077754   0.16181788
  -1.0074868 ]
 [-0.8822341  -2.265207    1.260047   ...  0.3029611   0.6034097
  -0.9613488 ]]


In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.36321402]], shape=(1, 1), dtype=float32)


In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 3
batch_size = 256
#steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = (len(x_train) // batch_size) * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 9e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'pooled_output': (N 9591041     preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=x_train,
                               y = y_train,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1
Epoch 1/3
56403/56403 [==============================] - 4444s 79ms/step - loss: 0.3171 - binary_accuracy: 0.7006
Epoch 2/3
56403/56403 [==============================] - 4476s 79ms/step - loss: 0.3128 - binary_accuracy: 0.7007
Epoch 3/3
56403/56403 [==============================] - 4470s 79ms/step - loss: 0.3128 - binary_accuracy: 0.7007


In [ ]:
classifier_model.save('/content/gdrive/MyDrive/job/Upskill/test2/saved_model/simple_bert_model')

In [ ]:
predictions = classifier_model.predict(x_test, batch_size = 256, verbose=1)

381/381 [==============================] - 694s 2s/step


In [ ]:
print(tf.sigmoid(predictions).numpy())
sigmoid_prediction = tf.sigmoid(predictions).numpy().reshape(predictions.shape[0])
print(sigmoid_prediction.shape)

[[0.06053815]
 [0.09717955]
 [0.09484228]
 ...
 [0.07923362]
 [0.16896898]
 [0.09524685]]
(97320,)


In [ ]:
submission = pd.DataFrame.from_dict({
    'id': test_df.id,
    'prediction': sigmoid_prediction
})

In [ ]:
submission.to_csv('/content/gdrive/MyDrive/job/Upskill/test2/submission.csv', index=False)

###DistilBERT model

Helper functions

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    
 
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer(text_chunk, padding = True, truncation=True, return_tensors="np")
        all_ids.extend(encs["input_ids"])
    
    return np.array(all_ids)

In [ ]:
def build_model(transformer, max_len=512):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

TPU configs (Optional)

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.119.209.162:8470


REPLICAS:  8


In [ ]:
# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

Create fast tokenizer

In [ ]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# Save the loaded tokenizer locally
#tokenizer.save_pretrained('/content/gdrive/MyDrive/job/Upskill/test2/.')
# Reload it with the huggingface tokenizers library
#fast_tokenizer = BertWordPieceTokenizer('/content/gdrive/MyDrive/job/Upskill/test2/vocab.txt', lowercase=False)
#fast_tokenizer

Loading  text data into memory

In [ ]:
x = x_train
y = y_train
y_aux = y_aux_train

In [ ]:
x_train_temp = x[:round(len(x) * 0.9)]
x_val_temp = x[round(len(x) * 0.9):]
y_train = y[:round(len(y) * 0.9)]
y_val = y[round(len(x) * 0.9):]
x_test_temp = x_test.copy()

In [ ]:
x_train = fast_encode(x_train_temp.astype(str), tokenizer, maxlen=MAX_LEN)
x_val = fast_encode(x_val_temp.astype(str), tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(x_test_temp.astype(str), tokenizer, maxlen=MAX_LEN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print(type(x_train))
print(x_train.shape)
print(x_train_temp[0])
print(x_train_temp.shape)
print(x_train[0])

<class 'numpy.ndarray'>
(1624387,)
This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!
(1624387,)
[ 101 2023 2003 2061 4658 1012 2009 1005 1055 2066 1010 1005 2052 2017
 2215 2115 2388 2000 3191 2023 1029 1029 1005 2428 2307 2801 1010 2092
 2589  999  102    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0


In [ ]:
x_train_hhhhh = x_train.copy()

In [ ]:
x_train_h2 = []
for i in range(len(x_train_hhhhh)):
  x_train_h2.append(x_train_hhhhh[i].numpy())

x_train_h2 = np.asarray(x_train_h2)

x_train_h2 = tf.convert_to_tensor(x_train_h2)

KeyboardInterrupt: ignored

In [ ]:
x_train = np.asarray(x_train)

In [ ]:
x_train = tf.convert_to_tensor(x_train)

ValueError: ignored

Build datasets objects

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

ValueError: ignored

In [ ]:
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-uncased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist TFBaseModelOutput(last_hi 66362880  
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 66,363,649
Trainable params: 66,363,649
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
train_history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = BATCH_SIZE,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    shuffle = True,
    workers = 8
)

ValueError: ignored